# Systematic Descriptor Modification Analysis

This notebook analyzes the systematic effects of individual modifications to SIFT descriptors, allowing us to isolate the contribution of each enhancement.

## Analysis Framework:
1. **Baseline Establishment**: Pure SIFT performance
2. **Single Modification Effects**: Individual impact of each change
3. **Combination Effects**: How modifications interact
4. **Performance Attribution**: Which modifications contribute most to accuracy gains

## Modifications Analyzed:
- **Color**: Grayscale (SIFT) vs Color (RGBSIFT)
- **Pooling**: None vs Domain-Size Pooling vs Stacking
- **Normalization**: L1 vs L2 norm


In [ ]:
# Import required libraries
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("Set1")
plt.rcParams['figure.figsize'] = (12, 8)

# Database connection
DB_PATH = "../../build/experiments.db"

In [ ]:
# Load systematic analysis data
def load_systematic_analysis_data():
    conn = sqlite3.connect(DB_PATH)
    
    query = """
    SELECT 
        e.id as experiment_id,
        e.descriptor_type,
        e.pooling_strategy,
        e.dataset_name,
        e.similarity_threshold,
        e.max_features,
        r.mean_average_precision,
        r.true_map_macro,
        r.true_map_micro,
        r.true_map_macro_with_zeros,
        r.true_map_micro_with_zeros,
        r.legacy_mean_precision,
        r.precision_at_1,
        r.precision_at_5,
        r.recall_at_1,
        r.recall_at_5,
        r.total_matches,
        r.total_keypoints,
        r.processing_time_ms,
        r.metadata
    FROM experiments e 
    JOIN results r ON e.id = r.experiment_id
    ORDER BY r.mean_average_precision DESC
    """
    
    df = pd.read_sql_query(query, conn)
    conn.close()
    
    # Use primary columns directly, with fallback to legacy for older records
    df['macro_map'] = df['true_map_macro'].fillna(df['mean_average_precision'])
    df['micro_map'] = df['true_map_micro'].fillna(df['mean_average_precision'])
    df['macro_map_with_zeros'] = df['true_map_macro_with_zeros']
    df['micro_map_with_zeros'] = df['true_map_micro_with_zeros']
    
    return df

# Parse descriptor configurations
def parse_descriptor_config(descriptor_name):
    """Extract configuration from descriptor names"""
    config = {
        'base_type': 'SIFT',
        'uses_color': False,
        'normalization': 'L2',
        'pooling': 'None'
    }
    
    name_lower = descriptor_name.lower()
    
    # Base type
    if 'rgbsift' in name_lower or 'color' in name_lower:
        config['base_type'] = 'RGBSIFT'
        config['uses_color'] = True
    
    # Normalization
    if 'l1' in name_lower:
        config['normalization'] = 'L1'
    
    return config

# Load and process data
df_systematic = load_systematic_analysis_data()

# Parse configurations
config_df = pd.DataFrame([parse_descriptor_config(name) for name in df_systematic['descriptor_type']])
df_systematic = pd.concat([df_systematic, config_df], axis=1)

# Clean pooling strategy names
pooling_map = {
    'none': 'None',
    'domain_size_pooling': 'DSP',
    'stacking': 'Stacking'
}
df_systematic['pooling_clean'] = df_systematic['pooling_strategy'].map(pooling_map).fillna(df_systematic['pooling_strategy'])
df_systematic['pooling'] = df_systematic['pooling_clean']

print(f"Loaded {len(df_systematic)} experiments for systematic analysis")
print("Available MAP metrics (using primary columns):")
print(f"- Primary MAP: {df_systematic['mean_average_precision'].min():.4f} - {df_systematic['mean_average_precision'].max():.4f}")
print(f"- Macro MAP: {df_systematic['macro_map'].min():.4f} - {df_systematic['macro_map'].max():.4f}")
print(f"- Micro MAP: {df_systematic['micro_map'].min():.4f} - {df_systematic['micro_map'].max():.4f}")
df_systematic.head(10)

## Baseline Performance Analysis

In [ ]:
# Establish baseline performance using macro MAP
baseline_condition = (df_systematic['base_type'] == 'SIFT') & \
                    (df_systematic['pooling'] == 'None') & \
                    (df_systematic['normalization'] == 'L2')

baseline_performance = df_systematic[baseline_condition]['macro_map'].iloc[0] if baseline_condition.any() else 0.36

print(f"=== BASELINE PERFORMANCE ===")
print(f"Baseline configuration: SIFT + Grayscale + No Pooling + L2 Norm")
print(f"Baseline Macro MAP: {baseline_performance:.4f}")

# Calculate improvement over baseline for all configurations
df_systematic['map_improvement'] = df_systematic['macro_map'] - baseline_performance
df_systematic['map_improvement_pct'] = (df_systematic['map_improvement'] / baseline_performance) * 100

print(f"\nTop 5 performing configurations:")
# Use dropna to handle NaN values in macro_map
valid_data = df_systematic.dropna(subset=['macro_map'])
top_5 = valid_data.nlargest(5, 'macro_map')[['descriptor_type', 'pooling_clean', 'macro_map', 'map_improvement_pct']]
for i, row in top_5.iterrows():
    print(f"{row['descriptor_type']} ({row['pooling_clean']}): Macro MAP = {row['macro_map']:.4f} (+{row['map_improvement_pct']:.1f}%)")

## Individual Modification Effects

In [ ]:
# Analyze individual modification effects
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Individual Modification Effects on Descriptor Performance', fontsize=16, fontweight='bold')

# Filter valid data
valid_data = df_systematic.dropna(subset=['macro_map'])

# 1. Color effect (SIFT vs RGBSIFT)
color_effect = valid_data.groupby(['uses_color', 'pooling', 'normalization'])['macro_map'].mean().reset_index()
color_comparison = color_effect.groupby('uses_color')['macro_map'].mean()

if len(color_comparison) >= 2:
    axes[0,0].bar(['Grayscale (SIFT)', 'Color (RGBSIFT)'], color_comparison.values, 
                 color=['lightblue', 'salmon'], alpha=0.8)
    for i, v in enumerate(color_comparison.values):
        axes[0,0].text(i, v + 0.005, f'{v:.3f}', ha='center', fontweight='bold')
else:
    # Only one color type available
    color_name = 'Color (RGBSIFT)' if color_comparison.index[0] else 'Grayscale (SIFT)'
    axes[0,0].bar([color_name], color_comparison.values, color=['lightblue'], alpha=0.8)
    axes[0,0].text(0, color_comparison.values[0] + 0.005, f'{color_comparison.values[0]:.3f}', ha='center', fontweight='bold')

axes[0,0].set_title('Color vs Grayscale Effect', fontweight='bold')
axes[0,0].set_ylabel('Macro MAP')

# 2. Pooling strategy effect
pooling_effect = valid_data.groupby('pooling')['macro_map'].mean().sort_values(ascending=False)
colors = ['lightcoral', 'lightgreen', 'lightskyblue'][:len(pooling_effect)]
bars = axes[0,1].bar(pooling_effect.index, pooling_effect.values, color=colors, alpha=0.8)
axes[0,1].set_title('Pooling Strategy Effect', fontweight='bold')
axes[0,1].set_ylabel('Macro MAP')
axes[0,1].set_xlabel('Pooling Strategy')
for i, v in enumerate(pooling_effect.values):
    axes[0,1].text(i, v + 0.005, f'{v:.3f}', ha='center', fontweight='bold')

# 3. Normalization effect
norm_effect = valid_data.groupby('normalization')['macro_map'].mean()
norm_colors = ['lightsteelblue', 'lightcoral'][:len(norm_effect)]
axes[1,0].bar(norm_effect.index, norm_effect.values, color=norm_colors, alpha=0.8)
axes[1,0].set_title('Normalization Effect', fontweight='bold')
axes[1,0].set_ylabel('Macro MAP')
axes[1,0].set_xlabel('Normalization Type')
for i, v in enumerate(norm_effect.values):
    axes[1,0].text(i, v + 0.005, f'{v:.3f}', ha='center', fontweight='bold')

# 4. Improvement magnitude comparison
improvements = {}

# Color improvement
if len(color_comparison) >= 2:
    improvements['Color'] = (color_comparison[True] - color_comparison[False]) / baseline_performance * 100

# Pooling improvements
if 'DSP' in pooling_effect.index and 'None' in pooling_effect.index:
    improvements['DSP'] = (pooling_effect['DSP'] - pooling_effect['None']) / baseline_performance * 100

if 'Stacking' in pooling_effect.index and 'None' in pooling_effect.index:
    improvements['Stacking'] = (pooling_effect['Stacking'] - pooling_effect['None']) / baseline_performance * 100

# Normalization improvement
if len(norm_effect) >= 2:
    if 'L1' in norm_effect.index and 'L2' in norm_effect.index:
        improvements['L1 Norm'] = (norm_effect['L1'] - norm_effect['L2']) / baseline_performance * 100

# Filter out missing values and sort by improvement
improvements = {k: v for k, v in improvements.items() if not pd.isna(v)}
improvements = dict(sorted(improvements.items(), key=lambda x: x[1], reverse=True))

if improvements:
    colors = ['green' if v > 0 else 'red' for v in improvements.values()]
    bars = axes[1,1].bar(improvements.keys(), improvements.values(), color=colors, alpha=0.7)
    axes[1,1].set_title('Performance Improvement by Modification', fontweight='bold')
    axes[1,1].set_ylabel('Improvement over Baseline (%)')
    axes[1,1].set_xlabel('Modification Type')
    axes[1,1].axhline(y=0, color='black', linestyle='-', alpha=0.3)
    for i, (k, v) in enumerate(improvements.items()):
        axes[1,1].text(i, v + (0.2 if v > 0 else -0.2), f'{v:.1f}%', ha='center', fontweight='bold')
else:
    axes[1,1].text(0.5, 0.5, 'Insufficient data\nfor comparison', ha='center', va='center', 
                   transform=axes[1,1].transAxes, fontsize=12)
    axes[1,1].set_title('Performance Improvement by Modification', fontweight='bold')

plt.tight_layout()
plt.show()

# Print numerical results
print("\n=== INDIVIDUAL MODIFICATION EFFECTS ===")
if improvements:
    for modification, improvement in improvements.items():
        print(f"{modification}: {improvement:+.1f}% change vs baseline")
else:
    print("Insufficient data for individual modification analysis")
    print("Available configurations:")
    for i, row in valid_data.iterrows():
        color_str = "Color" if row['uses_color'] else "Grayscale"
        print(f"  {color_str} + {row['pooling']} + {row['normalization']}: {row['macro_map']:.4f}")

## Combination Effects Analysis

In [ ]:
# Analyze combination effects and interactions
print("=== COMBINATION EFFECTS ANALYSIS ===")

# Create a comprehensive combination analysis
combination_analysis = df_systematic.groupby(['uses_color', 'pooling', 'normalization']).agg({
    'mean_average_precision': ['mean', 'count'],
    'map_improvement_pct': 'mean',
    'processing_time_ms': 'mean'
}).round(4)

combination_analysis.columns = ['MAP', 'Count', 'Improvement_%', 'ProcessTime_ms']
combination_analysis = combination_analysis.reset_index().sort_values('MAP', ascending=False)

print("\nTop performing combinations:")
print(combination_analysis.head(10))

# Visualize combination effects with interactive plot
fig = go.Figure()

# Create traces for different combinations
for i, row in combination_analysis.iterrows():
    color_str = "Color" if row['uses_color'] else "Gray"
    hover_text = f"{color_str} + {row['pooling']} + {row['normalization']}<br>" \
                f"MAP: {row['MAP']:.4f}<br>" \
                f"Improvement: {row['Improvement_%']:+.1f}%<br>" \
                f"Processing: {row['ProcessTime_ms']:.0f}ms"
    
    fig.add_trace(go.Scatter(
        x=[row['ProcessTime_ms']/1000],  # Convert to seconds
        y=[row['MAP']],
        mode='markers',
        marker=dict(
            size=15,
            color=row['Improvement_%'],
            colorscale='RdYlGn',
            showscale=True,
            colorbar=dict(title="Improvement %")
        ),
        text=hover_text,
        hovertemplate='%{text}<extra></extra>',
        name=f"{color_str}+{row['pooling']}+{row['normalization']}",
        showlegend=False
    ))

fig.update_layout(
    title='Descriptor Configuration Performance vs Processing Time',
    xaxis_title='Processing Time (seconds)',
    yaxis_title='Mean Average Precision',
    width=800,
    height=600
)

fig.show()

# Analyze synergistic effects
print("\n=== SYNERGISTIC EFFECTS ===")
print("Expected vs Actual performance for combinations:")

# Calculate expected performance if effects were purely additive
for i, row in combination_analysis.head(5).iterrows():
    expected_improvement = 0
    
    # Add individual effects
    if row['uses_color']:
        expected_improvement += improvements.get('Color', 0)
    
    if row['pooling'] == 'DSP':
        expected_improvement += improvements.get('DSP', 0)
    elif row['pooling'] == 'Stacking':
        expected_improvement += improvements.get('Stacking', 0)
    
    if row['normalization'] == 'L1':
        expected_improvement += improvements.get('L1 Norm', 0)
    
    expected_map = baseline_performance * (1 + expected_improvement/100)
    actual_map = row['MAP']
    synergy = actual_map - expected_map
    
    color_str = "Color" if row['uses_color'] else "Gray"
    config_name = f"{color_str}+{row['pooling']}+{row['normalization']}"
    
    print(f"{config_name}:")
    print(f"  Expected MAP: {expected_map:.4f}")
    print(f"  Actual MAP: {actual_map:.4f}")
    print(f"  Synergy: {synergy:+.4f} ({synergy/baseline_performance*100:+.1f}%)")
    print()

## Performance Attribution Analysis

In [ ]:
# Create performance attribution waterfall chart
import plotly.graph_objects as go

# Get the best performing configuration
best_config = combination_analysis.iloc[0]
best_map = best_config['MAP']

print(f"=== PERFORMANCE ATTRIBUTION ===")
print(f"Best configuration: {'Color' if best_config['uses_color'] else 'Gray'} + {best_config['pooling']} + {best_config['normalization']}")
print(f"Best MAP: {best_map:.4f} vs Baseline: {baseline_performance:.4f}")
print(f"Total improvement: {(best_map - baseline_performance)/baseline_performance*100:.1f}%")

# Create waterfall data
waterfall_data = [
    {'category': 'Baseline\n(SIFT+Gray+None+L2)', 'value': baseline_performance, 'type': 'absolute'},
]

# Add contributions from best config
running_total = baseline_performance

if best_config['uses_color']:
    color_contribution = improvements.get('Color', 0) / 100 * baseline_performance
    waterfall_data.append({'category': '+Color\n(RGBSIFT)', 'value': color_contribution, 'type': 'relative'})
    running_total += color_contribution

if best_config['pooling'] != 'None':
    pooling_contribution = improvements.get(best_config['pooling'], 0) / 100 * baseline_performance
    waterfall_data.append({'category': f'+{best_config["pooling"]}\nPooling', 'value': pooling_contribution, 'type': 'relative'})
    running_total += pooling_contribution

if best_config['normalization'] != 'L2':
    norm_contribution = improvements.get('L1 Norm', 0) / 100 * baseline_performance
    waterfall_data.append({'category': f'+{best_config["normalization"]}\nNorm', 'value': norm_contribution, 'type': 'relative'})
    running_total += norm_contribution

# Add synergy effect
synergy = best_map - running_total
if abs(synergy) > 0.001:
    waterfall_data.append({'category': 'Synergy\nEffects', 'value': synergy, 'type': 'relative'})

waterfall_data.append({'category': 'Final\nPerformance', 'value': best_map, 'type': 'absolute'})

# Create waterfall chart
categories = [item['category'] for item in waterfall_data]
values = [item['value'] for item in waterfall_data]
types = [item['type'] for item in waterfall_data]

# Calculate cumulative values for plotting
cumulative = []
running = 0
for i, (val, typ) in enumerate(zip(values, types)):
    if typ == 'absolute':
        cumulative.append(val)
        running = val
    else:
        cumulative.append(running + val)
        running += val

fig = go.Figure(go.Waterfall(
    name="Performance Attribution",
    orientation="v",
    measure=["absolute"] + ["relative"] * (len(categories)-2) + ["total"],
    x=categories,
    text=[f"{v:.4f}" for v in values],
    y=values,
    connector={"line":{"color":"rgb(63, 63, 63)"}},
))

fig.update_layout(
    title="Performance Attribution: From Baseline to Best Configuration",
    yaxis_title="Mean Average Precision",
    xaxis_title="Configuration Components",
    showlegend=False,
    width=800,
    height=500
)

fig.show()

# Print attribution breakdown
print("\nPerformance attribution breakdown:")
total_improvement = best_map - baseline_performance
for item in waterfall_data[1:-1]:  # Skip baseline and final
    contribution_pct = (item['value'] / total_improvement) * 100 if total_improvement != 0 else 0
    print(f"{item['category']}: {item['value']:+.4f} MAP ({contribution_pct:.1f}% of total improvement)")

In [ ]:
# Export systematic analysis results
output_dir = Path("../outputs")
output_dir.mkdir(exist_ok=True)

# Save comprehensive analysis data
df_systematic.to_csv(output_dir / "systematic_modification_analysis.csv", index=False)

# Save combination analysis
combination_analysis.to_csv(output_dir / "descriptor_combination_analysis.csv", index=False)

# Save individual modification effects
improvement_df = pd.DataFrame(list(improvements.items()), columns=['Modification', 'Improvement_Pct'])
improvement_df.to_csv(output_dir / "individual_modification_effects.csv", index=False)

# Create summary report
with open(output_dir / "systematic_analysis_summary.txt", 'w') as f:
    f.write("SYSTEMATIC DESCRIPTOR MODIFICATION ANALYSIS\n")
    f.write("=" * 50 + "\n\n")
    
    f.write(f"BASELINE PERFORMANCE:\n")
    f.write(f"Configuration: SIFT + Grayscale + No Pooling + L2 Norm\n")
    f.write(f"MAP: {baseline_performance:.4f}\n\n")
    
    f.write(f"BEST CONFIGURATION:\n")
    best_desc = "Color" if best_config['uses_color'] else "Grayscale"
    f.write(f"Configuration: {best_desc} + {best_config['pooling']} + {best_config['normalization']}\n")
    f.write(f"MAP: {best_map:.4f}\n")
    f.write(f"Total Improvement: {(best_map - baseline_performance)/baseline_performance*100:.1f}%\n\n")
    
    f.write("INDIVIDUAL MODIFICATION EFFECTS:\n")
    for mod, imp in improvements.items():
        f.write(f"{mod}: {imp:+.1f}%\n")

print(f"\n=== ANALYSIS COMPLETE ===")
print(f"Results exported to: {output_dir}")
print(f"\nKey Insights:")
print(f"- Baseline MAP: {baseline_performance:.3f}")
print(f"- Best MAP: {best_map:.3f} (+{(best_map-baseline_performance)/baseline_performance*100:.1f}%)")
print(f"- Most effective modification: {max(improvements.items(), key=lambda x: x[1])[0]} (+{max(improvements.values()):.1f}%)")
print(f"- Experiments analyzed: {len(df_systematic)}")
print(f"- Unique configurations: {len(combination_analysis)}")